In [ ]:
# === Mount Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

# ==============================
# DAWN MODULE
# ==============================
class DAWN(nn.Module):
    """
    DAWN module: Learns IPC-like filters as a trainable preprocessing layer.
    This module takes raw images and produces augmented feature maps that
    highlight edges/structures at multiple scales.
    """
    def __init__(self, in_channels: int):
        super(DAWN, self).__init__()
        self.in_channels = in_channels
        # fine-scale edge detection
        self.conv_fine = nn.Conv2d(in_channels, 1, kernel_size=3, padding=1)
        # coarse-scale structure detection
        self.conv_coarse = nn.Conv2d(in_channels, 1, kernel_size=3, padding=4, dilation=4)
        # weight initialization
        nn.init.kaiming_normal_(self.conv_fine.weight, nonlinearity='relu')
        nn.init.constant_(self.conv_fine.bias, 0.0)
        nn.init.kaiming_normal_(self.conv_coarse.weight, nonlinearity='relu')
        nn.init.constant_(self.conv_coarse.bias, 0.0)
        self.out_channels = in_channels + 2

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        fine = F.relu(self.conv_fine(x))
        coarse = F.relu(self.conv_coarse(x))
        out = torch.cat([x, fine, coarse], dim=1)
        return out


# ==============================
# U-NET
# ==============================
class DoubleConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(DoubleConv, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


class UNet(nn.Module):
    def __init__(self, in_channels: int, out_classes: int = 1):
        super(UNet, self).__init__()
        features = [64, 128, 256, 512]
        self.enc_blocks = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        prev_channels = in_channels
        for feat in features:
            self.enc_blocks.append(DoubleConv(prev_channels, feat))
            prev_channels = feat
        self.bottleneck = DoubleConv(prev_channels, prev_channels * 2)
        bottleneck_channels = prev_channels * 2  # store for clarity
        self.dec_blocks = nn.ModuleList()
        self.upconvs = nn.ModuleList()
        for feat in reversed(features):
            # use bottleneck_channels (1024) for the first upconv, then halve each time
            self.upconvs.append(nn.ConvTranspose2d(bottleneck_channels, feat, kernel_size=2, stride=2))
            self.dec_blocks.append(DoubleConv(feat * 2, feat))
            bottleneck_channels = feat  # update for next iteration

        self.final_conv = nn.Conv2d(features[0], out_classes, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        enc_features = []
        for enc_block in self.enc_blocks:
            x = enc_block(x)
            enc_features.append(x)
            x = self.pool(x)
        x = self.bottleneck(x)
        for i, dec_block in enumerate(self.dec_blocks):
            skip_feat = enc_features[-(i+1)]
            x = self.upconvs[i](x)
            if x.shape[2:] != skip_feat.shape[2:]:
                diffY = skip_feat.size(2) - x.size(2)
                diffX = skip_feat.size(3) - x.size(3)
                x = F.pad(x, [diffX // 2, diffX - diffX//2, diffY // 2, diffY - diffY//2])
            x = torch.cat([skip_feat, x], dim=1)
            x = dec_block(x)
        logits = self.final_conv(x)
        return logits


# ==============================
# COMBINED MODEL
# ==============================
class DawnUNet(nn.Module):
    def __init__(self, in_channels: int = 3, out_classes: int = 1, use_dawn: bool = True):
        super(DawnUNet, self).__init__()
        self.use_dawn = use_dawn
        if use_dawn:
            self.dawn = DAWN(in_channels)
            self.unet = UNet(in_channels=self.dawn.out_channels, out_classes=out_classes)
        else:
            self.dawn = None
            self.unet = UNet(in_channels=in_channels, out_classes=out_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.use_dawn:
            x = self.dawn(x)
        logits = self.unet(x)
        return logits


# ==============================
# DATASET
# ==============================
class SegmentationDataset(Dataset):
    def __init__(self, images_dir: str, masks_dir: str, transform_image=None, transform_mask=None):
        self.image_files = sorted(os.listdir(images_dir))
        self.mask_files = sorted(os.listdir(masks_dir))
        assert len(self.image_files) == len(self.mask_files)
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform_image = transform_image
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir, self.mask_files[idx])
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        if self.transform_image:
            image = self.transform_image(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)
        mask = (mask > 0.5).float()
        return image, mask


# ==============================
# TRAINING AND VISUALIZATION
# ==============================
def main():
    # Drive dataset paths
    train_images = "/content/drive/MyDrive/research/unet-ipc-data/DIS-TR/im-good"
    train_masks = "/content/drive/MyDrive/research/unet-ipc-data/DIS-TR/gt-good"

    transform_img = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    transform_mask = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

    train_dataset = SegmentationDataset(train_images, train_masks, transform_img, transform_mask)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DawnUNet(in_channels=3, out_classes=1, use_dawn=True).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.BCEWithLogitsLoss()

    model.train()
    for epoch in range(20):
        total_loss, total_dice, total_iou = 0.0, 0.0, 0.0

        for batch_idx, (images, masks) in enumerate(train_loader):
            print(f"  → Training batch {batch_idx+1}/{len(train_loader)}")
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            logits = model(images)
            loss = loss_fn(logits, masks)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * images.size(0)

            # --- Dice & IoU for this batch ---
            with torch.no_grad():
                probs = torch.sigmoid(logits)
                preds = (probs > 0.5).float()
                intersection = (preds * masks).sum(dim=(1,2,3))
                union = preds.sum(dim=(1,2,3)) + masks.sum(dim=(1,2,3))
                dice = ((2 * intersection + 1e-7) / (union + 1e-7)).mean().item()
                iou = ((intersection + 1e-7) / (union - intersection + 1e-7)).mean().item()

            total_dice += dice * images.size(0)
            total_iou += iou * images.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        avg_dice = total_dice / len(train_loader.dataset)
        avg_iou = total_iou / len(train_loader.dataset)

        print(f"Epoch {epoch+1}/20 - Loss: {avg_loss:.4f}, Dice: {avg_dice:.3f}, IoU: {avg_iou:.3f}")

    # Visualization
    os.makedirs("output_viz", exist_ok=True)
    model.eval()
    with torch.no_grad():
        for idx, (img, mask) in enumerate(train_loader):
            img, mask = img.to(device), mask.to(device)
            pred = torch.sigmoid(model(img))
            pred = (pred > 0.5).float()
            img_np = img[0].cpu().permute(1, 2, 0).numpy()
            pred_np = pred[0, 0].cpu().numpy()
            mask_np = mask[0, 0].cpu().numpy()
            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            axs[0].imshow(img_np)
            axs[0].set_title("Input Image")
            axs[1].imshow(pred_np, cmap="gray")
            axs[1].set_title("Predicted Mask")
            axs[2].imshow(mask_np, cmap="gray")
            axs[2].set_title("Ground Truth")
            for ax in axs:
                ax.axis("off")
            plt.tight_layout()
            plt.savefig(f"output_viz/sample_{idx}.png")
            plt.show()
            plt.close()
            if idx >= 2:
                break

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  → Training batch 1/750
  → Training batch 2/750
  → Training batch 3/750
  → Training batch 4/750
  → Training batch 5/750
  → Training batch 6/750
  → Training batch 7/750
  → Training batch 8/750
  → Training batch 9/750
  → Training batch 10/750
  → Training batch 11/750
  → Training batch 12/750
  → Training batch 13/750
  → Training batch 14/750
  → Training batch 15/750
  → Training batch 16/750
  → Training batch 17/750
  → Training batch 18/750
  → Training batch 19/750


In [2]:
!nvidia-smi


Sun Oct 19 23:59:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             35W /   70W |    2450MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----